In [6]:
import json

# Load the JSON data from data.json
with open('data/custom.json') as f:
    data = json.load(f)



In [7]:
food_names = list(map(lambda f: f['name'], data['foods']))
food_names

['Cooked buckwheat',
 'Cooked Chicken Breast',
 'Olive oil',
 'Fish, raw, wild, Atlantic, salmon',
 'Yogurt',
 'Orange']

In [8]:
nutrients = {}
nutrients_types = set()
for food in data['foods']:
    print(food)
    food_name = food['name']
    nutrients[food_name] = {}
    for nutrient in food['nutrients']:
        nutrients[food_name][nutrient] = food['nutrients'][nutrient]['value']
        nutrients_types.add(nutrient)

nutrients

{'name': 'Cooked buckwheat', 'nutrients': {'calories': {'value': 92, 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_cooked%2C_roasted_nutritional_value.html?size=100%20g', 'https://www.yazio.com/en/foods/buckwheat-cooked.html']}, 'carbohydrates': {'value': 20, 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_cooked%2C_roasted_nutritional_value.html?size=100%20g', 'https://www.yazio.com/en/foods/buckwheat-cooked.html']}, 'protein': {'value': 3.4, 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_cooked%2C_roasted_nutritional_value.html?size=100%20g']}, 'fat': {'value': 0.6, 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_cooked%2C_roasted_nutritional_value.html?size=100%20g']}, 'vitamin_d': {'value': 0, 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_cooked%2C_roasted_nutritional_value.html?size=100%20g']}, 'iron': {'value': 0.8, 'measure': 'mg', 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_

{'Cooked buckwheat': {'calories': 92,
  'carbohydrates': 20,
  'protein': 3.4,
  'fat': 0.6,
  'vitamin_d': 0,
  'iron': 0.8,
  'calcium': 7,
  'vitamin_c': 0},
 'Cooked Chicken Breast': {'calories': 151,
  'protein': 29,
  'carbohydrates': 0,
  'fat': 3,
  'vitamin_d': 0.1,
  'iron': 0.9,
  'calcium': 13,
  'vitamin_c': 0},
 'Olive oil': {'calories': 884,
  'protein': 0,
  'carbohydrates': 0,
  'fat': 100,
  'vitamin_d': 0,
  'iron': 0.6,
  'calcium': 1,
  'vitamin_c': 0},
 'Fish, raw, wild, Atlantic, salmon': {'calories': 142,
  'protein': 20,
  'carbohydrates': 0,
  'fat': 6.3,
  'vitamin_d': 11,
  'iron': 0.8,
  'calcium': 12,
  'vitamin_c': 0},
 'Yogurt': {'calories': 46,
  'protein': 2.7,
  'carbohydrates': 4.2,
  'fat': 2,
  'vitamin_d': 0,
  'iron': 0.1,
  'calcium': 183,
  'vitamin_c': 0.8},
 'Orange': {'calories': 47,
  'protein': 0.9,
  'carbohydrates': 12,
  'fat': 0.1,
  'vitamin_d': 0,
  'iron': 0.1,
  'calcium': 40,
  'vitamin_c': 53.2}}

In [9]:
# nutrients['Cooked buckwheat groats'].get('calories', 0)

In [10]:
import pulp as pl
prob = pl.LpProblem("Meal_Planning_Problem", pl.LpMinimize)
food_vars = pl.LpVariable.dicts("Amount", food_names, lowBound=0)
prob += pl.lpSum([food_vars[f] for f in food_names]), "Total Weight of Food"

calories = 2500
prob += pl.lpSum([(nutrients[f].get('calories', 0) * food_vars[f]) for f in food_names] 
                    ) >= calories, "Min Calories"

def add_nutrient_constraints(prob, nutrients, food_vars, nutrient_name, min_value, max_value):
    if nutrient_name not in nutrients_types:
        raise ValueError(f"Nutrient {nutrient_name} not found in the data")
    # Adding constraints for minimum nutrient values, if specified
    if min_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) >= min_value, f"Min {nutrient_name}"
    if max_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) <= max_value, f"Max {nutrient_name}"

add_nutrient_constraints(prob, nutrients, food_vars, 'carbohydrates', calories * 0.45 / 4,  calories * 0.65 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'protein', calories * 0.1 / 4,  calories * 0.35 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'fat', calories * 0.0 / 9,  calories * 0.3 / 9)
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_d', 15, 100) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'iron', 20.5, 45) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'calcium', 1000, 2500) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_c', 90, 2000) # mg

prob.solve()
print("Status:", pl.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        formatted_name = v.name.replace("Amount_", "").replace("_", " ")
        print(f"{formatted_name} = {v.varValue * 100} grams")

Status: Optimal
Cooked Chicken Breast = 232.40101 grams
Cooked buckwheat = 1849.652 grams
Fish, raw, wild, Atlantic, salmon = 381.18392 grams
Orange = 163.1801 grams
Yogurt = 398.52356000000003 grams



3. Micronutrients (vitamins and minerals):
   [x] a. Vitamin D: Vital for bone health, immune function, and calcium absorption.
   [x] b. Iron: Essential for oxygen transport in the body and preventing anemia.
   [x] c. Calcium: Crucial for strong bones, teeth, and muscle function.
   [ ]d. Vitamin C: Important for immune function, collagen production, and antioxidant properties.
   e. Vitamin A: Necessary for vision, immune function, and cell growth.
   f. B Vitamins (B12, B6, Folate, etc.): Essential for energy production, brain function, and red blood cell formation.
   g. Magnesium: Vital for muscle and nerve function, energy production, and bone health.
   h. Potassium: Important for heart function, muscle contraction, and blood pressure regulation.
   i. Zinc: Crucial for immune function, wound healing, and protein synthesis.

4. Fiber: Promotes digestive health, regulates blood sugar, and aids in weight management.

5. Antioxidants: Help protect cells from damage caused by free radicals, reducing the risk of chronic diseases.

6. Phytochemicals: Plant compounds that offer various health benefits, such as reducing inflammation and supporting heart health.

7. Omega-3 fatty acids: Important for brain function, heart health, and reducing inflammation.

8. Probiotics: Beneficial bacteria that support gut health and immune function.

While all of these components are important for a well-rounded, healthy diet, the most vital aspects are staying hydrated, consuming a balance of macronutrients, and ensuring an adequate intake of essential vitamins and minerals.